# Power Point Presentation maker

## Configs

In [1]:
import os

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    msg = "Missing API_KEY"
    raise OSError(msg)

In [ ]:
client = OpenAI(api_key=api_key)
# completion = client.beta.chat.completions.parse(
#     model=
# )